In [1]:
import random
import numpy as np
import pandas as pd 
import networkx as nx 
from tqdm import tqdm
from gensim.models import Word2Vec

from alias import alias_sample, create_alias_tables

In [2]:
def gene_transition_prob(G, p, q):
    """
    1. 生成节点之间的转移概率
    2. 生成与采样节点上一个节点距离相关的采样概率
    """
    node_alias_dict = {}
    # 遍历所有节点, 计算节点之间的转移概率
    for node in G.nodes:
        # 取出当前节点的所有一阶邻居节点，及对应的权重、
        # G[node][nbr]返回的是一个字典{'weight': score}
        nbr_node_weight_list = [G[node][nbr].get('weight', 1.0) \
                                for nbr in G.neighbors(node)] 
        # 归一化概率
        normlized_prob_list = [float(weight) / sum(nbr_node_weight_list) \
                               for weight in nbr_node_weight_list]
        # 构建alias表
        node_alias_dict[node] = create_alias_tables(normlized_prob_list)
        
        
    edge_alias_dict = {}
    # 遍历所有的边，计算跟距离相关的采样节点之间的采样概率
    for edge in G.edges:
        t, v = edge[0], edge[1]
        unnormalized_prob_list = []
        # 遍历节点v的所有邻居节点
        for x in G.neighbors(v):
            # 取出v和x之间的权重
            weight = G[v][x].get('weight', 1.0)
            # 判断节点v邻居节点x与节点t的距离
            if x == t: # d_tx=0
                unnormalized_prob_list.append(weight / p)
            elif G.has_edge(t, x): # d_tx=1
                unnormalized_prob_list.append(weight)
            else:
                unnormalized_prob_list.append(weight / q)
        # 概率归一化
        normlized_prob_list = [float(weight) / sum(unnormalized_prob_list) \
                               for weight in unnormalized_prob_list]
        
        # 构建alias表
        edge_alias_dict[(t, v)] = create_alias_tables(normlized_prob_list)
        
    return node_alias_dict, edge_alias_dict

In [3]:
def node2vec_walk(G, walk_length=10, p=0.25, q=4):
    """
    1. 生成每个节点，每条边，对应的邻居采样概率分布的alias采样表
    2. 遍历所有节点，以每个节点为起始节点开始采样，直到当前节点没有邻居节点
       或采样的序列达到设定的长度
    """
    # 获取图中所有节点、边的Alias采样表
    node_alias_dict, edge_alias_dict = gene_transition_prob(G, p, q)    
    
    sentences_list = []
    
    nodes_list = G.nodes
    for node in tqdm(nodes_list):
        # 以每个节点为起始节点
        tmp_sentence_list = [node]
        
        while len(tmp_sentence_list) < walk_length:
            cur_node = tmp_sentence_list[-1]
            nbrs_cur_list = list(G.neighbors(cur_node))
            
            # 如果当前节点没有邻居节点了退出当前序列的采样
            if len(nbrs_cur_list) == 0:
                break
                
            # 如果是起始节点，使用节点采样，因为它没有上一个节点，无法使用距离采样
            if len(tmp_sentence_list) == 1:
                # 通过alias采样表来采样
                accept_prob = node_alias_dict[cur_node][0]
                alias_tables = node_alias_dict[cur_node][1]
            else:
                # 根据不同的距离，来采样
                # 获取当前采样节点及上一个节点
                t, v = tmp_sentence_list[-2], tmp_sentence_list[-1]
                accept_prob = edge_alias_dict[(t,v)][0]
                alias_tables = edge_alias_dict[(t,v)][1]
                
            sample_index = alias_sample(accept_prob, alias_tables)
            tmp_sentence_list.append(nbrs_cur_list[sample_index])
            
        sentences_list.append(tmp_sentence_list)
        
    return sentences_list

In [4]:
params_dict = {
    'sentences': '',
    'vector_size': 32,
    'sg': 1,
    'min_count': 0,
    'window': 10,
    'negative': 5,
    'workers': 10,
    'epochs': 10
}

G = nx.read_edgelist('./graph.csv', create_using=nx.DiGraph(), \
                     nodetype=None, data=[('weight', int)])
sentences = node2vec_walk(G)

params_dict['sentences'] = sentences

# 定义word2vec模型
w2v_model = Word2Vec(**params_dict)

100%|████████████████████████████████████████████████████████████████████████████████████| 31364/31364 [00:00<00:00, 52597.99it/s]
